### Convert RGBA Array to Bitmap Image (Spectrogram)

In [3]:
def array_to_bitmap(rgba_array, out_basename):

    if rgba_array.ndim != 3 or rgba_array.shape[2] != 4:
        raise ValueError("Expected an array for RGBA data")

    # Convert to uint8 
    rgba_uint8 = (rgba_array * 255).astype(np.uint8)

    # Create image and save as bitmap
    bmp_filename = f"{out_basename}.bmp"
    img = Image.fromarray(rgba_uint8, mode="RGBA")
    img.save(bmp_filename)


### Divide Large IQ Sample Input into Spectrogram Dictated Sample Counts

In [11]:
def IQsample_load(file, c):
    # Number of spectrograms to be generated
    num_spectrograms = math.floor(os.path.getsize(file) / (c * 8))

    # Folder name based on c
    if c == 33024:
        folder_name = '256x256'
    elif c == 65664:
        folder_name = '256x512'
    elif c == 131328:
        folder_name = '512x512'
    elif c == 262400:
        folder_name = '512x1024'
    elif c == 524800:
        folder_name = '1024x1024'
    else:
        raise ValueError(f"Unknown c value: {c}")

    # Ensure folder exists
    os.makedirs(folder_name, exist_ok=True)

    # Read and save samples
    offset = 0
    for counter in range(num_spectrograms):
        # Read one block of IQ samples
        iq_sample = np.fromfile(file, dtype=np.complex64, count=c, offset=offset)

        # Save it to its own binary file
        out_path = os.path.join(folder_name, f'sample_{counter}.bin')
        iq_sample.tofile(out_path)

        # Update offset (8 bytes per complex64 sample)
        offset += c * 8

Spectrogram SciPy:

* spectrogram(time_domain_input, sampling_freq, nsperg, noverlap, return_onesided)
    * nperseg = # of samples per segment
    * noverlap = number of samples to overlap between segments
    * return_onesided = True (returns a two-sided spectrogram with real and complex)

| Spectrogram Dimensions | nperseg | noverlap | Samples Needed|
|:------:|:------:|:------:|:------:|
| 256x256 | 256 | 128 | 33,024 |
| 256x512 | 256 | 128 | 65,664 |
| 512x512 | 512 | 256 | 131,328 |
| 512x1024 | 512 | 256 | 262,400 |
| 1024x1024 | 1024 | 512 | 524,800 |

In [6]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.signal import spectrogram
from PIL import Image

# === CONFIG ===
post_filename = "Post-Channel/post_channel.iq"
pre_filename = "Pre-Channel/pre_channel.iq"
fs = 23.04e6

pre_out_image = "spectrogram_pre.png"
post_out_image = "spectrogram_post.png"

# Create individual Pre-Channel IQ files for each spectrogram
IQsample_load(pre_filename, 65664)
IQsample_load(pre_filename, 131328)
IQsample_load(pre_filename, 262400)
IQsample_load(pre_filename, 524800)

# Create individual Post-Channel IQ files for each spectrogram
IQsample_load(post_filename, 65664)
IQsample_load(post_filename, 131328)
IQsample_load(post_filename, 262400)
IQsample_load(post_filename, 524800)

2048
170
348160
348160


/local_scratch/slurm.6686851/ipykernel_890814/1551677329.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  color_map = cm.get_cmap('viridis')


In [ ]:
# === COMPUTE SPECTROGRAM ===
# Smag is power spectral density (amplitude squared)
f_pre, t_pre, pre_Smag = spectrogram(pre_iq, fs=fs, nperseg=n, noverlap=512, return_onesided=False)
f_post, t_post, post_Smag = spectrogram(post_iq, fs=fs, nperseg=n, noverlap=512, return_onesided=False)

print(f_pre.size)
print(t_pre.size)
print(pre_Smag.size)
print(post_Smag.size)
# Convert power to power in decibels
pre_Smag_dB = 10 * np.log10(pre_Smag + 1e-12) # +1e-12 prevents infinite log when spectrogram power = 0
post_Smag_dB = 10 * np.log10(post_Smag + 1e-12)

In [ ]:
# min and max values mapped to endpoints of colormap
# 0db = full scale power, weak components -100db or less
vmin, vmax = -120, 0 
norm = colors.Normalize(vmin=vmin, vmax=vmax, clip=True)
color_map = cm.get_cmap('viridis')
# Map normalized data (RGBA floats in [R, G, B, A] values between 0-1)
rgba_pre = color_map(norm(pre_Smag_dB))
rgba_post = color_map(norm(post_Smag_dB))

# Create a bitmap image from these RGBA values for pre and post-channel spectrograms
array_to_bitmap(rgba_pre, "spectrogram_pre")
array_to_bitmap(rgba_post, "spectrogram_post")

In [ ]:
# === PLOT CLEAN SPECTROGRAM ===
height_px = 1024
width_px = 512
plt.figure(figsize=(6, 4), frameon=False)
plt.axis("off")             # remove axes
plt.imshow(pre_Smag_dB, aspect='auto', origin='lower', cmap='viridis')
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # remove all padding
plt.margins(0, 0)
plt.savefig(pre_out_image, dpi=300, bbox_inches='tight', pad_inches=0)
plt.close()

plt.figure(figsize=(6, 4), frameon=False)
plt.axis("off")             # remove axes
plt.imshow(post_Smag_dB, aspect='auto', origin='lower', cmap='viridis')
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # remove all padding
plt.margins(0, 0)
plt.savefig(post_out_image, dpi=300, bbox_inches='tight', pad_inches=0)
plt.close()